In [ ]:
%run 0_helper_functions.ipynb

In [ ]:
IN_DIR="../IN"
IN_DB=f"{IN_DIR}/stealer.sqlite3"
OUT_DIR="../OUT"

In [ ]:
!echo $IN_DIR && ls -lah $IN_DIR
!echo $OUT_DIR && ls -lah $OUT_DIR 

## Open DB


In [ ]:
conn = sqlite3.connect(IN_DB)

In [ ]:
conn

## Read in request files

In [ ]:
mail_domains = []
url_domains = []
with open(f"{IN_DIR}/mail_domains.txt", "r") as file:
    mail_domains = [line.strip() for line in file]
with open(f"{IN_DIR}/url_domains.txt", "r") as file:
    url_domains = [line.strip() for line in file]

In [ ]:
mail_domains

In [ ]:
url_domains

## Email domains

In [ ]:
records = []
count = 1
for apex in mail_domains:
    print("{} / {}   {}                                                                                                            ".format(
        count,len(mail_domains), apex), end="\r")
    count = count + 1
    for row in conn.execute("""
        SELECT s.user, s.passwd, s.url, s.user_apex, s.file, f.timestamp
        FROM stealer s
        LEFT JOIN files f ON s.file = f.name
        WHERE s.user_apex = ? COLLATE NOCASE;
    """, (apex, ) ).fetchall() :
        records.append(row)
email_df = pd.DataFrame(records, columns = [ "username", "masked_passwd", "url", "user_apex", "file_ref", "scraped_date" ])

In [ ]:
records

In [ ]:
email_df

In [ ]:
email_df.to_csv(f"{OUT_DIR}/email_records.csv", index=False)

## URL domains

In [ ]:
records = []
count = 1
for apex in url_domains:
    print("{} / {}   {}                                                                                                            ".format(
        count,len(url_domains), apex), end="\r")
    count = count + 1
    for row in conn.execute("""
        SELECT s.user, s.passwd, s.url, s.url_apex, s.file, f.timestamp
        FROM stealer s
        LEFT JOIN files f ON s.file = f.name
        WHERE s.url_apex = ? COLLATE NOCASE;
    """, (apex, ) ).fetchall() :
        records.append(row)
url_df = pd.DataFrame(records, columns = [ "username", "masked_passwd", "url", "url_apex", "file_ref", "scraped_date" ])

In [ ]:
url_df

In [ ]:
url_df.to_csv(f"{OUT_DIR}/url_records.csv", index=False)

In [ ]:
ls -lah $OUT_DIR 